<a href="https://colab.research.google.com/github/nitish6121999/assignment_text_from_pdf/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import sqlite3
from tabulate import tabulate

def create_database():
    conn = sqlite3.connect('articles.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS articles (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            author TEXT,
            publication_date TEXT NOT NULL,
            content TEXT NOT NULL
        )
    ''')
    conn.commit()
    conn.close()

def scrape_articles():
    url = 'https://indianexpress.com/'
    response = requests.get(url)
    if response.status_code != 200:
        print("Failed to retrieve the website")
        return

    soup = BeautifulSoup(response.content, 'html.parser')
    articles = []

    for link in soup.find_all('a', href=True):
        if '/business/' in link['href']:
            article_url = link['href']
            if not article_url.startswith('https://indianexpress.com'):
                article_url = 'https://indianexpress.com' + article_url

            article_response = requests.get(article_url)
            if article_response.status_code != 200:
                continue

            article_soup = BeautifulSoup(article_response.content, 'html.parser')

            # Extract title
            title_tag = article_soup.find('h1', class_='native_story_title')
            if title_tag:
                title = title_tag.get_text(strip=True)
            else:
                print(f"Title not found for {article_url}")
                continue

            # Extract author
            author = None
            author_patterns = ['written by', 'by']
            for element in article_soup.find_all(['span', 'div', 'p']):
                text = element.get_text(strip=True).lower()
                for pattern in author_patterns:
                    if text.startswith(pattern):
                        author = re.sub(rf'{pattern}', '', text, flags=re.I).strip()
                        break
                if author:
                    break

            # Extract publication date
            pub_date_tag = article_soup.find('meta', {'name': 'publish-date'})
            pub_date = pub_date_tag['content'] if pub_date_tag else datetime.now().isoformat()

            # Extract content
            content = ' '.join(p.get_text(strip=True) for p in article_soup.find_all('p'))
            if not content:
                print(f"Content not found for {article_url}")
                continue

            articles.append((title, author, pub_date, content))

    return articles

def store_articles(articles):
    conn = sqlite3.connect('articles.db')
    cursor = conn.cursor()

    cursor.executemany('''
        INSERT INTO articles (title, author, publication_date, content)
        VALUES (?, ?, ?, ?)
    ''', articles)

    conn.commit()
    conn.close()

def show_tables():
    conn = sqlite3.connect('articles.db')
    cursor = conn.cursor()

    # List all tables
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print("Tables in the database:")
    print(tabulate(tables, headers=["Table Name"], tablefmt="grid"))

    # Describe the articles table
    cursor.execute("PRAGMA table_info(articles);")
    columns = cursor.fetchall()
    print("\nSchema of the 'articles' table:")
    print(tabulate(columns, headers=["CID", "Name", "Type", "NotNull", "DefaultVal", "PK"], tablefmt="grid"))

    # Retrieve and display data from the articles table with pagination
    def fetch_rows(offset=0, limit=10):
        cursor.execute("SELECT * FROM articles LIMIT ? OFFSET ?", (limit, offset))
        return cursor.fetchall()

    offset = 0
    limit = 10
    while True:
        rows = fetch_rows(offset, limit)
        if not rows:
            break
        print("\nData in the 'articles' table (page {}):".format(offset // limit + 1))
        print(tabulate(rows, headers=["ID", "Title", "Author", "Publication Date", "Content"], tablefmt="grid"))
        offset += limit
        user_input = input("Press Enter to see more, or type 'q' to quit: ")
        if user_input.lower() == 'q':
            break

    conn.close()

def main():
    create_database()
    articles = scrape_articles()
    if articles:
        store_articles(articles)
        print(f"Stored {len(articles)} articles in the database.")
    else:
        print("No articles found or an error occurred.")

    show_tables()

if __name__ == '__main__':
    main()


Title not found for https://indianexpress.com/section/business/
Title not found for https://indianexpress.com/section/business/
Title not found for https://indianexpress.com/article/business/market/stock-market-today-live-updates-nse-nifty-50-bse-sensex-gift-nifty-rupee-dollar-9362950/
Title not found for https://indianexpress.com/article/business/market/stock-market-today-live-updates-nse-nifty-50-bse-sensex-gift-nifty-rupee-dollar-9362950/
Title not found for https://indianexpress.com/article/business/market/stock-market-today-live-updates-nse-nifty-50-bse-sensex-gift-nifty-rupee-dollar-9362950/
Title not found for https://indianexpress.com/section/business/
Title not found for https://indianexpress.com/section/business/
Title not found for https://indianexpress.com/section/business/
Stored 20 articles in the database.
Tables in the database:
+-----------------+
| Table Name      |
+=================+
| articles        |
+-----------------+
| sqlite_sequence |
+-----------------+

Sc